In [1]:
import shapeworks as sw
import glob
import pyvista as pv
import vtk
import numpy as np
import pyvista as pv
import shapeworks as sw
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
DATASET_PATH = "DATA/RF_FULGUR"
OUTPUT_PATH = DATASET_PATH + "mirrored"

In [3]:
all_images = glob.glob(DATASET_PATH + "/*.nii.gz")
images_to_process = []
for image_path in all_images:
    if image_path.__contains__("left"):
        images_to_process.append(image_path)
        
print("Processing", len(images_to_process), "images:")
for image_path in images_to_process:
    print(f'   {image_path}')
    
ref_image_path = images_to_process[0]
ref_image_right_path = all_images[1]

Processing 5 images:
   DATA/RF_FULGUR\FULGUR_002_283592_left_label_4.nii.gz
   DATA/RF_FULGUR\FULGUR_028_61082_left_label_4.nii.gz
   DATA/RF_FULGUR\FULGUR_029_15058_left_label_4.nii.gz
   DATA/RF_FULGUR\FULGUR_059_236946_left_label_4.nii.gz
   DATA/RF_FULGUR\FULGUR_076_46061_left_label_4.nii.gz


In [4]:
# Function to mirror the image along the x-axis
""" 
def mirror_image(image):
    image_data = image.toArray()
    mirrored_data = np.flip(image_data, axis=1)  # Flip along x-axis
    return sw.Image(mirrored_data.astype(np.float32))
"""

def mirror_image(image):
    # Get the image data as a numpy array
    image_data = image.toArray()
    
    # Flip the data along the x-axis (axis=0)
    mirrored_data = np.flip(image_data, axis=2)
    
    # Create a new Shapeworks Image from the flipped data
    mirrored_image = sw.Image(mirrored_data.astype(np.float32))
    
    # Copy the spacing from the original image
    mirrored_image.setSpacing(image.spacing())
    
    # Adjust the origin to account for the flip
    original_origin = image.origin()
    spacing = image.spacing()
    dims = image.dims()
    
    # Calculate the new origin
    new_origin = list(original_origin)
    new_origin[0] = original_origin[0] + (dims[0] - 1) * spacing[0]
    mirrored_image.setOrigin(new_origin)
    
    return mirrored_image


# Function to convert Shapeworks image to mesh
def image_to_mesh(image):
    # Convert image to numpy array
    image_data = image.toArray()
    # Calculate isovalue
    min_val = image_data.min()
    max_val = image_data.max()
    isovalue = (max_val + min_val) / 2.0
    # Generate the mesh
    return sw.sw2vtkMesh(image.toMesh(isovalue=isovalue))

In [5]:
image_left_path = DATASET_PATH + '/FULGUR_002_283592_left_label_4.nii.gz'
image_right_path = DATASET_PATH + '/FULGUR_002_283592_right_label_4.nii.gz'

In [6]:
# Load images
left_image = sw.Image(image_left_path)
right_image = sw.Image(image_right_path)

# Convert images to meshes
left_mesh = image_to_mesh(left_image)
right_mesh = image_to_mesh(right_image)

# Wrap meshes with PyVista
pv_left_mesh = pv.wrap(left_mesh)
pv_right_mesh = pv.wrap(right_mesh)

In [7]:
# Mirror the left image
mirrored_left_image = mirror_image(left_image)

# Convert to a mesh
mirrored_mesh = image_to_mesh(mirrored_left_image)

# Wrap the mesh
pv_mirrored_mesh = pv.wrap(mirrored_mesh)

In [8]:
# Function to plot a single mesh in an external PyVista window
def plot_mesh(mesh, title, color):
    plotter = pv.Plotter(title=title, notebook=False)
    plotter.add_mesh(mesh, color=color, show_edges=False)
    plotter.add_axes()
    plotter.add_bounding_box()
    plotter.show()
    
# Plot Left Original and Mirrored Left together in one external window
def plot_two_meshes(mesh1, mesh2, title, color1, color2):
    plotter = pv.Plotter(title=title, notebook=False)
    plotter.add_mesh(mesh1, color=color1, show_edges=False, label="Left Original")
    plotter.add_mesh(mesh2, color=color2, show_edges=False, label="Mirrored Left")
    plotter.add_axes()
    plotter.add_bounding_box()
    plotter.add_legend()
    plotter.show()

# Plot each mesh separately
#plot_mesh(pv_left_mesh, "Left Original Muscle", "blue")
#plot_mesh(pv_mirrored_mesh, "Mirrored Left Muscle", "green")
#plot_mesh(pv_right_mesh, "Right Original Muscle", "red")

plot_two_meshes(pv_left_mesh, pv_mirrored_mesh, "Left Original and Mirrored Left", "red", "green")

In [9]:
plot_two_meshes(pv_right_mesh, pv_mirrored_mesh, "Mirrored Left and Right", "red", "green")